In [103]:
import numpy as np
import pandas as pd
import pandas.io.sql as psql
from pandas import ExcelWriter
from pandas import ExcelFile
import openpyxl
pd.options.mode.chained_assignment = None

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn import model_selection, linear_model
from sklearn.feature_selection import RFE

import statsmodels.api as sm


plt.style.use("ggplot")


In [104]:
performance_2019 = pd.read_excel('../../capstone2/data/2019_sales_by_month.xlsx')
performance_2019.head()

,Property Name,Property Code,Brand,#Rooms,Management Company,Registration Date,Activation Date,Last Transaction,Revenue,Profit Margin,Gross Profit,Pend Prod,Neg Inv,Low Prod Sales,Shift Rep Days,PMS Port,Column1,Month of Reporting
0,Hilton Garden Inn Times Square,NYCMW,Hilto,369.0,"Highgate Hotels, Inc.",2017-10-06,2017-10-27,2019-02-01 03:49:00,48190.59,0.72970,35164.64,76.0,119.0,NaN,a long time,35010.0,Details Use Archive,January
1,InterContinental New Orleans,MSYHA,Inter,484.0,Dimension Development Company,2018-08-17,2018-09-20,2019-02-01 05:39:00,22829.80,0.67301,15364.72,69.0,86.0,99.0,6 months,5011.0,Details Use Archive,January
2,DoubleTree Suites by Hilton Hotel Boston - Cam...,BOSSB,Doubl,308.0,Hilton Worldwide,2018-04-10,2018-06-11,2019-02-01 04:58:00,22488.70,0.69884,15715.96,36.0,165.0,NaN,6 months,35010.0,Details Use Archive,January
3,Hilton Garden Inn New Orleans Convention Center,MSYGI,Hilto,286.0,Interstate Hotels and Resorts,2018-06-27,2018-09-04,2019-02-01 05:52:00,15917.05,0.84608,13467.05,142.0,239.0,NaN,7 days,35010.0,Details Use Archive,January
4,Crowne Plaza Fredericton-Lord Beaverbrook,YFCQS,Crown,169.0,IHG - Crowne Plaza,2018-09-26,2019-02-11,2019-03-01 03:58:00,544.45,0.56147,305.69,NaN,1.0,NaN,about 21 hours,5011.0,Details Use Archive,February


In [105]:
#drop NaN, drop unwanted columns, create additional column 'Flag'
cleaning_df = performance_2019.dropna(subset=['Property Code'])
cleaning_df.drop(['Column1', 'Pend Prod', 'Neg Inv', 'Low Prod Sales', 'Shift Rep Days', 'PMS Port', 'Last Transaction', 'Registration Date'], axis=1, inplace=True)
cleaning_df['Flag']=cleaning_df['Brand']

In [106]:
#replace month strings with int
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
value = 1
for month in months:
    cleaning_df.replace(to_replace=month, value=value, inplace=True)
    value+=1

In [107]:
#clean up brand field 
cleaning_df['Brand'].replace(to_replace=['Tru B', 'TRU B', 'TRU b', 'Tru b', 'Home2', 'Homew', 'The S', 'Hampt', 'Doubl', 'Embas', 'Miram'], value='Hilto', inplace=True)
cleaning_df['Brand'].replace(to_replace=['Stayb', 'Inter', 'IHG A', 'Holid', 'Avid ', 'Crown', 'Candl'], value='IHG', inplace=True)
cleaning_df['Brand'].replace(to_replace= ['Aston', 'Delta', 'Renai'], value='Marri', inplace=True)
cleaning_df['Brand'].replace(to_replace= ['Quali', 'Comfo'], value='Choic', inplace=True)
cleaning_df['Brand'].replace(to_replace= 'La Quin', value='Wyndh', inplace=True)
cleaning_df['Brand'].replace(to_replace= ['Blueb', 'River'], value='Apart', inplace=True)
cleaning_df['Brand'].replace(to_replace= ['Platt'], value='Impul', inplace=True)
cleaning_df['Brand'].replace(to_replace= ['Hammo'], value='Indep', inplace=True)

prop_mask1 = (cleaning_df['Property Code'] == 'LAXMA') | (cleaning_df['Property Code'] == 'LGBMY')
cleaning_df['Brand'][prop_mask1] = 'Hilto'
prop_mask2 = cleaning_df['Property Code'] == 'IHG - ATLID'
cleaning_df['Brand'][prop_mask2] = 'IHG'

In [109]:
#clean up flag field
cleaning_df['Flag'].replace(to_replace=['TRU B', 'TRU b', 'Tru B', 'Tru b'], value='Tru by Hilton', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'The S', value='Tapestry Collections', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Homew', value='Homewood Suites by Hilton', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Home2', value='Home2 Suites by Hilton', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'The S', value='Tapestry Collections', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Holid', value='Holiday Inn', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Stayb', value='Staybridge Suites', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Crown', value='Crowne Plaza', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Embas', value='Embassy Hilton', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Doubl', value='DoubleTree by Hilton', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Candl', value='Candlewood Suites', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Avid ', value='Avid', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Comfo', value='Comfort Suites', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Quali', value='Quality Suites', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'IHG A', value='IHG Army Hotels', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'La Quinta', value='La Quinta Inn & Suites', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Inter', value='Intercontinental Hotels', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Renai', value='Renaissance', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Delta', value='Delta Hotels', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Best ', value='Best Western Plus', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Miram', value='Curio Collection', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Hammo', value='Independant', inplace=True)
cleaning_df['Flag'].replace(to_replace=['Blueb', 'River'], value='Apartment Complex', inplace=True)
cleaning_df['Flag'].replace(to_replace= 'Platt', value='Retail', inplace=True)

In [111]:
unique_brands = cleaning_df['Brand'].unique()
print(cleaning_df['Brand'].value_counts())
print(unique_brands)

Hilto    3967
IHG       467
Choic      23
Marri      19
Apart      12
La Qu      12
Best        4
Indep       1
Impul       1
Name: Brand, dtype: int64
['Hilto' 'IHG' 'La Qu' 'Choic' 'Marri' 'Apart' 'Best ' 'Impul' 'Indep']


In [112]:
#drop all hotels with activation date after 2019-01-01
age_mask = cleaning_df['Activation Date'] <= '2019-01-01'

#drop all hotels wtih a profit margin > 0.74 or < 0.44
high_profit_mask = cleaning_df['Profit Margin'] < 0.74
low_profit_mask = cleaning_df['Profit Margin'] > 0.44
drop_masks = cleaning_df[age_mask & high_profit_mask & low_profit_mask].copy()
drop_masks['Brand'].value_counts()

Hilto    2316
IHG       204
La Qu      12
Choic      11
Marri       7
Name: Brand, dtype: int64

In [121]:
#create a list of brands with low reporting
drop_brands = []
for brand in unique_brands:
    drop_brand_count = drop_masks[drop_masks['Brand'] == brand]
    if len(drop_brand_count) <= 12:
        drop_brands.append(brand)

print(drop_brands)
#drop all brands with less than 12 months reported

low_count_mask = ~drop_masks['Brand'].isin(drop_brands)

dropped_everything = drop_masks[low_count_mask].copy()
dropped_everything['Brand'].value_counts()

['La Qu', 'Choic', 'Marri', 'Apart', 'Best ', 'Impul', 'Indep']


Hilto    2316
IHG       204
Name: Brand, dtype: int64

In [125]:
#create one-hot-encoded columns for Brands
hilton_mask = drop_masks['Brand'] == 'Hilto'
drop_masks['is_hilton'] = drop_masks['Brand'] == 'Hilto'
# IHG_mask = drop_masks['Brand'] == 'IHG'
# la_quinta_mask = drop_masks['Brand'] == 'La Qu'
# choice_mask = drop_masks['Brand'] == 'Choic'
# marriott_mask = drop_masks['Brand'] == 'Marri'
drop_masks


,Property Name,Property Code,Brand,#Rooms,Management Company,Activation Date,Revenue,Profit Margin,Gross Profit,Month of Reporting,Flag,is_hilton
0,Hilton Garden Inn Times Square,NYCMW,Hilto,369.0,"Highgate Hotels, Inc.",2017-10-27,48190.59,0.72970,35164.64,1,Hilto,True
1,InterContinental New Orleans,MSYHA,IHG,484.0,Dimension Development Company,2018-09-20,22829.80,0.67301,15364.72,1,Intercontinental Hotels,False
2,DoubleTree Suites by Hilton Hotel Boston - Cam...,BOSSB,Hilto,308.0,Hilton Worldwide,2018-06-11,22488.70,0.69884,15715.96,1,DoubleTree by Hilton,True
5,Crowne Plaza Lansing West,LANCP,IHG,212.0,IHG - Crowne Plaza,2018-10-26,611.00,0.60129,367.39,4,Crowne Plaza,False
6,Homewood Suites Savannah Historic District,SAVRB,Hilto,162.0,"North Point Hospitality Group, Inc.",2018-04-10,14439.60,0.63231,9130.27,1,Homewood Suites by Hilton,True
...,...,...,...,...,...,...,...,...,...,...,...,...
4477,Hampton Inn Magnolia,AGOSN,Hilto,68.0,Shinn Enterprises,2018-04-03,835.85,0.60038,501.83,12,Hampt,True
4481,Tru by Hilton Bryan College Station,CLLCS,Hilto,98.0,"Integral Hospitality Solutions, LLC",2018-08-27,809.84,0.67184,544.08,12,Tru by Hilton,True
4487,Comfort Suites Downtown Orlando,FL094,Choic,122.0,ONIX Hospitality,2017-07-13,762.75,0.58116,443.28,12,Comfort Suites,False
4498,Hampton Inn Greensboro-East,GSOEA,Hilto,78.0,We Care Hotels,2018-09-29,614.38,0.54211,333.06,12,Hampt,True
